# 3 Classificação e previsão 
## 3.1) Quão maior os gastos e investimentos com a educação, menor será a taxa de analfabetismo?

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

## 3.1.1) Carga e transformação da massa bruta de todos os indicadores e países.

In [2]:
massa_bruta_pais_por_indicadores = pd.read_csv('../data/massa_bruta_pais_por_indicadores.csv')

anos = ['1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', 
        '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']

indicadores = ['UIS.LP.AG15T24', 'UIS.ILLPOP.AG25T64', 'UIS.LP.AG65',
               'UIS.XSPENDP.FDPUB.FNS', 'UIS.XSPENDP.FDPUB.FNCAP', 'UIS.XSPENDP.FDPUB.FNNONS', 
               'UIS.XGDP.FSGOV.FDINSTADM.FFD', 'SE.XPD.TOTL.GD.ZS']

paises = massa_bruta_pais_por_indicadores['Country Name'].unique()

df_massa_bruta_por_indicador = massa_bruta_pais_por_indicadores.loc[massa_bruta_pais_por_indicadores['Indicator Code'].isin(indicadores)]

colunas_transformacao = ['Pais', 'Indicador', '1998-1999', '1999-2000', '2000-2001', '2001-2002', 
                        '2002-2003', '2003-2004', '2004-2005', '2005-2006', '2006-2007', '2007-2008', 
                        '2008-2009', '2009-2010', '2010-2011', '2011-2012']
df_resultado_transformacao = pd.DataFrame(columns=colunas_transformacao)

colunas_transformacao_teste = ['Pais', 'Indicador', '2012-2013']
df_resultado_transformacao_teste = pd.DataFrame(columns=colunas_transformacao_teste)

for pais in paises:
    df_pais = df_massa_bruta_por_indicador.loc[df_massa_bruta_por_indicador['Country Name'] == pais]

    df_1998_a_2013 = df_pais.iloc[:, 2:18]
    df_1998_a_2013.index = range(df_1998_a_2013.shape[0])
    df_1998_a_2013 = df_1998_a_2013.fillna(0)
   
    indicador = df_pais['Indicator Code']
    indicador.index = range(indicador.shape[0])
    
    for index, indicador in indicador.items():
        valor_1998_1999 = 1 if df_1998_a_2013['1999'].iloc[index] - df_1998_a_2013['1998'].iloc[index] > 0 else 0
        valor_1999_2000 = 1 if df_1998_a_2013['2000'].iloc[index] - df_1998_a_2013['1999'].iloc[index] > 0 else 0
        valor_2000_2001 = 1 if df_1998_a_2013['2001'].iloc[index] - df_1998_a_2013['2000'].iloc[index] > 0 else 0
        valor_2001_2002 = 1 if df_1998_a_2013['2002'].iloc[index] - df_1998_a_2013['2001'].iloc[index] > 0 else 0
        valor_2002_2003 = 1 if df_1998_a_2013['2003'].iloc[index] - df_1998_a_2013['2002'].iloc[index] > 0 else 0
        valor_2003_2004 = 1 if df_1998_a_2013['2004'].iloc[index] - df_1998_a_2013['2003'].iloc[index] > 0 else 0
        valor_2004_2005 = 1 if df_1998_a_2013['2005'].iloc[index] - df_1998_a_2013['2004'].iloc[index] > 0 else 0
        valor_2005_2006 = 1 if df_1998_a_2013['2006'].iloc[index] - df_1998_a_2013['2005'].iloc[index] > 0 else 0
        valor_2006_2007 = 1 if df_1998_a_2013['2007'].iloc[index] - df_1998_a_2013['2006'].iloc[index] > 0 else 0
        valor_2007_2008 = 1 if df_1998_a_2013['2008'].iloc[index] - df_1998_a_2013['2007'].iloc[index] > 0 else 0
        valor_2008_2009 = 1 if df_1998_a_2013['2009'].iloc[index] - df_1998_a_2013['2008'].iloc[index] > 0 else 0
        valor_2009_2010 = 1 if df_1998_a_2013['2010'].iloc[index] - df_1998_a_2013['2009'].iloc[index] > 0 else 0
        valor_2010_2011 = 1 if df_1998_a_2013['2011'].iloc[index] - df_1998_a_2013['2010'].iloc[index] > 0 else 0
        valor_2011_2012 = 1 if df_1998_a_2013['2012'].iloc[index] - df_1998_a_2013['2011'].iloc[index] > 0 else 0
        valor_2012_2013 = 1 if df_1998_a_2013['2013'].iloc[index] - df_1998_a_2013['2012'].iloc[index] > 0 else 0

        valores = [pais, indicador, valor_1998_1999, valor_1999_2000, valor_2000_2001, valor_2001_2002, 
                   valor_2002_2003, valor_2003_2004, valor_2004_2005, valor_2005_2006, valor_2006_2007, 
                   valor_2007_2008, valor_2008_2009, valor_2009_2010, valor_2010_2011, valor_2011_2012]
        
        df_auxiliar = pd.DataFrame([valores], columns=colunas_transformacao)
        df_auxiliar.index = range(df_auxiliar.shape[0])
        df_resultado_transformacao = pd.concat([df_auxiliar, df_resultado_transformacao])
        
        valores_teste = [pais, indicador, valor_2012_2013]
        df_auxiliar_teste = pd.DataFrame([valores_teste], columns=colunas_transformacao_teste)
        df_auxiliar_teste.index = range(df_auxiliar_teste.shape[0])
        df_resultado_transformacao_teste = pd.concat([df_auxiliar_teste, df_resultado_transformacao_teste])


df_resultado_transformacao = df_resultado_transformacao.sort_values(['Pais'], ascending=[True])
df_resultado_transformacao.index = range(df_resultado_transformacao.shape[0])

df_resultado_transformacao_teste = df_resultado_transformacao_teste.sort_values(['Pais'], ascending=[True])
df_resultado_transformacao_teste.index = range(df_resultado_transformacao_teste.shape[0])

## 3.1.2) Transformação e seleção para o indicador de analfabetismo entre 15 a 24 anos.

#### Realiza a listagem dos países que possuem pelo menos 50% dos valores preenchidos para o indicador de analfabetismo de 15 a 24 anos ('UIS.LP.AG15T24').

In [3]:
paises_na_media = []
df_indicador_15_a_24 = df_massa_bruta_por_indicador.loc[df_massa_bruta_por_indicador['Indicator Code'] == 'UIS.LP.AG15T24']
df_indicador_15_a_24.index = range(df_indicador_15_a_24.shape[0])
for index, row in df_indicador_15_a_24.iterrows():
    pais = row['Country Name']
#     media_zeros = row[2:18].isna().mean()
#     if media_zeros <= 0.5:
#         paises_na_media.append(pais)
    paises_na_media.append(pais)

df_paises_na_media = pd.DataFrame(columns=['Pais'], data=paises_na_media)

#### Transforma os indicadores, dos países selecionados acima, em colunas.  Isso é feito para o período entre 1998 e 2012.

In [4]:
indicadores_15_a_24 = ['UIS.LP.AG15T24', 'UIS.XSPENDP.FDPUB.FNS', 'UIS.XSPENDP.FDPUB.FNCAP', 
                       'UIS.XSPENDP.FDPUB.FNNONS', 'UIS.XGDP.FSGOV.FDINSTADM.FFD', 'SE.XPD.TOTL.GD.ZS']

df_analfabetismo_15_a_24 = pd.DataFrame()

for indicador in indicadores_15_a_24:
    df_indicador = df_resultado_transformacao.loc[df_resultado_transformacao['Indicador'] == indicador]
    df_indicador = df_indicador.loc[df_indicador['Pais'].isin(df_paises_na_media['Pais'])]
    df_indicador.index = range(df_indicador.shape[0])
    df_periodos_indicador = df_indicador.drop(['Pais', 'Indicador'], axis=1)    
    df_periodos_indicador = df_periodos_indicador.stack()
    df_periodos_indicador.index = range(df_periodos_indicador.shape[0])
    df_analfabetismo_15_a_24[indicador] = df_periodos_indicador.tolist()

nomes_colunas = {'UIS.LP.AG15T24': 'Analfabetismo 15 a 24', 
                 'UIS.ILLPOP.AG25T64': 'Analfabetismo 25 a 64', 
                 'UIS.LP.AG65': 'Analfabetismo 65+', 
                 'UIS.XSPENDP.FDPUB.FNS': 'Gastos Pessoas',
                 'UIS.XSPENDP.FDPUB.FNCAP': 'Gastos Totais', 
                 'UIS.XSPENDP.FDPUB.FNNONS': 'Gastos Logística',
                 'UIS.XGDP.FSGOV.FDINSTADM.FFD': 'Gastos Infraestrutura', 
                 'SE.XPD.TOTL.GD.ZS': 'Gastos Governo'}

df_analfabetismo_15_a_24 = df_analfabetismo_15_a_24.rename(columns = nomes_colunas)

#### Transforma os indicadores, dos mesmos países selecionados acima, em colunas. Isso é feito para o período entre 2012 e 2013 (dados de teste).

In [5]:
df_analfabetismo_15_a_24_teste = pd.DataFrame()

for indicador in indicadores_15_a_24:
    df_indicador_teste = df_resultado_transformacao_teste.loc[df_resultado_transformacao_teste['Indicador'] == indicador]
    df_indicador_teste = df_indicador_teste.loc[df_indicador_teste['Pais'].isin(df_paises_na_media['Pais'])]
    df_indicador_teste.index = range(df_indicador_teste.shape[0])
    
    df_periodos_indicador_teste = df_indicador_teste.drop(['Pais', 'Indicador'], axis=1)    
    df_periodos_indicador_teste = df_periodos_indicador_teste.stack()
    df_periodos_indicador_teste.index = range(df_periodos_indicador_teste.shape[0])
    df_analfabetismo_15_a_24_teste[indicador] = df_periodos_indicador_teste.tolist()

df_analfabetismo_15_a_24_teste = df_analfabetismo_15_a_24_teste.rename(columns = nomes_colunas)

#### Análise e previsão dos dados para a taxa de analfabetismo de 15 a 24 anos.

##### 1.Introdução à classificação

In [6]:
# Realiza a previsão separando os dados entre treinamento (todos os dados do período de 1998 a 2012) 
# e testes (todos os dados do período de 2012 a 2013).

#1998 a 2012
treino_x = df_analfabetismo_15_a_24.loc[:, ['Gastos Pessoas', 'Gastos Totais', 'Gastos Logística', 'Gastos Infraestrutura', 'Gastos Governo']]
treino_y = df_analfabetismo_15_a_24.loc[:, 'Analfabetismo 15 a 24']

#2012 a 2013
teste_x = df_analfabetismo_15_a_24_teste.loc[:, ['Gastos Pessoas', 'Gastos Totais', 'Gastos Logística', 'Gastos Infraestrutura', 'Gastos Governo']]
teste_y = df_analfabetismo_15_a_24_teste.loc[:, 'Analfabetismo 15 a 24']

modelo = LinearSVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)
taxa_de_acerto = accuracy_score(teste_y, previsoes)

print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))
print("Taxa de acerto: %.2f" % (taxa_de_acerto * 100))

#Geração da baseline para verificar se a acurácia foi boa ou ruim.
previsoes_baseline = np.zeros(217)
acuracia = accuracy_score(teste_y, previsoes_baseline)
print("A acurácia do algoritmo de baseline foi de %.2f%%" % (acuracia * 100))

Treinaremos com 3038 elementos e testaremos com 217 elementos
Taxa de acerto: 91.24
A acurácia do algoritmo de baseline foi de 91.24%


##### 2. Testes replicáveis, estratificação e lendo dados da internet

In [7]:
# A diferença entre esse cenário ao anterior, é que aqui a divisão da massa de testes é o número de registros.
# Ou seja, aqui foi utilizado 75% de todos os registros (101 elementos do período de 1998 a 2013) para 
# treinamento e 25% (34 elementos do período de 1998 a 2013) para testes.

#1998 a 2013
df_analfabetismo_15_a_24_total = df_analfabetismo_15_a_24.append(df_analfabetismo_15_a_24_teste)
x = df_analfabetismo_15_a_24_total[['Gastos Pessoas', 'Gastos Totais', 'Gastos Logística', 'Gastos Infraestrutura', 'Gastos Governo']]
y = df_analfabetismo_15_a_24_total['Analfabetismo 15 a 24']

treino_x = x[:2441]
treino_y = y[:2441]
teste_x = x[2441:]
teste_y = y[2441:]
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = LinearSVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)
acuracia = accuracy_score(teste_y, previsoes)
print("A acurácia foi de %.2f%%" % (acuracia * 100))

Treinaremos com 2441 elementos e testaremos com 814 elementos
A acurácia foi de 89.19%


In [8]:
# Divisão automática da massa de treinamento e testes. Isso foi feito através do "train_test_split" da lib "SKLearn"

#Define a ordem dos números aleatórios.
#Assim, tetira a aleatoriedade da separação dos dados de treino e teste.
SEED = 5
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, random_state = SEED, test_size=0.25, stratify=y)
#O parâmetro "stratify=y" orienta o algoritmo a estratificar (separar proporcionalmente) os dados de acordo com o "y".

print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = LinearSVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)
acuracia = accuracy_score(teste_y, previsoes)
print("A acurácia foi de %.2f%%" % (acuracia * 100))

#Geração da baseline para verificar se a acurácia foi boa ou ruim.
previsoes_baseline = np.zeros(814)
acuracia = accuracy_score(teste_y, previsoes_baseline)
print("A acurácia do algoritmo de baseline foi de %.2f%%" % (acuracia * 100))

Treinaremos com 2441 elementos e testaremos com 814 elementos
A acurácia foi de 88.94%
A acurácia do algoritmo de baseline foi de 88.94%


In [9]:
# Utilização do modelo "SVC"

#Define a ordem dos números aleatórios.
#Assim, tetira a aleatoriedade da separação dos dados de treino e teste.
SEED = 5
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, random_state = SEED, test_size=0.25, stratify=y)
#O parâmetro "stratify=y" orienta o algoritmo a estratificar (separar proporcionalmente) os dados de acordo com o "y".

print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

#Reescalando os dados para uma faixa similar.


modelo = SVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)
acuracia = accuracy_score(teste_y, previsoes)
print("A acurácia foi de %.2f%%" % (acuracia * 100))

#Geração da baseline para verificar se a acurácia foi boa ou ruim.
previsoes_baseline = np.zeros(814)
acuracia = accuracy_score(teste_y, previsoes_baseline)
print("A acurácia do algoritmo de baseline foi de %.2f%%" % (acuracia * 100))

Treinaremos com 2441 elementos e testaremos com 814 elementos
A acurácia foi de 88.94%
A acurácia do algoritmo de baseline foi de 88.94%


## 3.1.3) Transformação e seleção para o indicador de analfabetismo entre 25 a 64 anos.

In [10]:
indicadores_25_a_64 = ['UIS.ILLPOP.AG25T64', 'UIS.XSPENDP.FDPUB.FNS', 'UIS.XSPENDP.FDPUB.FNCAP', 
                       'UIS.XSPENDP.FDPUB.FNNONS', 'UIS.XGDP.FSGOV.FDINSTADM.FFD', 'SE.XPD.TOTL.GD.ZS']

df_analfabetismo_25_a_64 = pd.DataFrame()

## 3.1.4) Transformação e seleção para o indicador de analfabetismo para 65 anos ou mais.

In [11]:
indicadores_65 = ['UIS.LP.AG65', 'UIS.XSPENDP.FDPUB.FNS', 'UIS.XSPENDP.FDPUB.FNCAP', 
                  'UIS.XSPENDP.FDPUB.FNNONS', 'UIS.XGDP.FSGOV.FDINSTADM.FFD', 'SE.XPD.TOTL.GD.ZS']

df_analfabetismo_65 = pd.DataFrame()

## 3.1.5) Validação da eficiência da previsão

In [12]:
#6) Aplicar algoritmo de classificação e previsão


#Fatos:
#1) As linhas do dataframe estão agrupadas por países.
#2) Cada país possui uma lista de indicadores.
#3) A lista de indicadores é dividida em dois grupos:
#   Grupo 1 = indicadores de analfabetismo (influenciados): UIS.LP.AG15T24, UIS.ILLPOP.AG25T64, UIS.LP.AG65
#   Grupo 2 = indicadores econômicos (influenciadores): UIS.XSPENDP.FDPUB.FNS, UIS.XSPENDP.FDPUB.FNCAP, UIS.XSPENDP.FDPUB.FNNONS, UIS.XGDP.FSGOV.FDINSTADM.FFD, SE.XPD.TOTL.GD.ZS
#4) Além disso, existem as colunas (com e 1) da diferença entre os anos de 1998 a 2013.

#O quê fazer?
#1) Transformar o dataset em:
#   Linhas = período de anos (1998-1999, 1999-2000, ..., 2012-2013)
#   Colunas = um indicador por coluna: UIS.LP.AG15T24, UIS.ILLPOP.AG25T64, UIS.LP.AG65, UIS.XSPENDP.FDPUB.FNS, UIS.XSPENDP.FDPUB.FNCAP, UIS.XSPENDP.FDPUB.FNNONS, UIS.XGDP.FSGOV.FDINSTADM.FFD, SE.XPD.TOTL.GD.ZS


df_periodo_datas = df_resultado_transformacao.iloc[:, 2:18]


SEED = 20

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y,
                                                         random_state = SEED, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = LinearSVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 2441 elementos e testaremos com 814 elementos
A acurácia foi 88.94%


## 3.1.6) Carga e transformação da massa bruta dos dados com pelo menos 50% dos valores preenchidos.

Isso reduzirá a quantidade de elementos para 351. Na solução 3.1.1, sem a restrição de 50% do preenchimento, foram utilizados 1953 elementos.

In [13]:
massa_bruta_pais_por_indicadores = pd.read_csv('../data/massa_bruta_pais_por_indicadores.csv')

paises = massa_bruta_pais_por_indicadores['Country Name'].unique()

df_massa_bruta_por_indicador = massa_bruta_pais_por_indicadores.loc[massa_bruta_pais_por_indicadores['Indicator Code'].isin(indicadores)]

df_classificado = pd.DataFrame(columns=colunas_transformacao)

for pais in paises:
    df_pais = df_massa_bruta_por_indicador.loc[df_massa_bruta_por_indicador['Country Name'] == pais]

    df_1998_a_2013 = df_pais.iloc[:, 2:18]
    df_1998_a_2013.index = range(df_1998_a_2013.shape[0])
    media_valores_invalidos = df_1998_a_2013.isna().mean(axis=1).mean()
    
    if media_valores_invalidos < 0.5:
        
        df_1998_a_2013 = df_1998_a_2013.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).fillna(0)
        
        for index, value in indicador.items():            
            valor_1998_1999 = 1 if df_1998_a_2013['1999'].iloc[0] - df_1998_a_2013['1998'].iloc[0] > 0 else 0
            valor_1999_2000 = 1 if df_1998_a_2013['2000'].iloc[0] - df_1998_a_2013['1999'].iloc[0] > 0 else 0
            valor_2000_2001 = 1 if df_1998_a_2013['2001'].iloc[0] - df_1998_a_2013['2000'].iloc[0] > 0 else 0
            valor_2001_2002 = 1 if df_1998_a_2013['2002'].iloc[0] - df_1998_a_2013['2001'].iloc[0] > 0 else 0
            valor_2002_2003 = 1 if df_1998_a_2013['2003'].iloc[0] - df_1998_a_2013['2002'].iloc[0] > 0 else 0
            valor_2003_2004 = 1 if df_1998_a_2013['2004'].iloc[0] - df_1998_a_2013['2003'].iloc[0] > 0 else 0
            valor_2004_2005 = 1 if df_1998_a_2013['2005'].iloc[0] - df_1998_a_2013['2004'].iloc[0] > 0 else 0
            valor_2005_2006 = 1 if df_1998_a_2013['2006'].iloc[0] - df_1998_a_2013['2005'].iloc[0] > 0 else 0
            valor_2006_2007 = 1 if df_1998_a_2013['2007'].iloc[0] - df_1998_a_2013['2006'].iloc[0] > 0 else 0
            valor_2007_2008 = 1 if df_1998_a_2013['2008'].iloc[0] - df_1998_a_2013['2007'].iloc[0] > 0 else 0
            valor_2008_2009 = 1 if df_1998_a_2013['2009'].iloc[0] - df_1998_a_2013['2008'].iloc[0] > 0 else 0
            valor_2009_2010 = 1 if df_1998_a_2013['2010'].iloc[0] - df_1998_a_2013['2009'].iloc[0] > 0 else 0
            valor_2010_2011 = 1 if df_1998_a_2013['2011'].iloc[0] - df_1998_a_2013['2010'].iloc[0] > 0 else 0
            valor_2011_2012 = 1 if df_1998_a_2013['2012'].iloc[0] - df_1998_a_2013['2011'].iloc[0] > 0 else 0
            valor_2012_2013 = 1 if df_1998_a_2013['2013'].iloc[0] - df_1998_a_2013['2012'].iloc[0] > 0 else 0
            
            valores = [pais, value, valor_1998_1999, valor_1999_2000, valor_2000_2001, valor_2001_2002, 
                       valor_2002_2003, valor_2003_2004, valor_2004_2005, valor_2005_2006, valor_2006_2007, 
                       valor_2007_2008, valor_2008_2009, valor_2009_2010, valor_2010_2011, valor_2011_2012, 
                       valor_2012_2013]

            df_auxiliar = pd.DataFrame([valores], columns=colunas_transformacao)
            df_auxiliar.index = range(df_auxiliar.shape[0])
            df_classificado = pd.concat([df_auxiliar, df_classificado])
        
df_classificado = df_classificado.sort_values(['Pais'], ascending=[True])
df_classificado.index = range(df_classificado.shape[0])
df_classificado

AttributeError: 'str' object has no attribute 'items'

## 3.1.2.1) Validação da eficiência da previsão

In [ ]:
df_classificado.describe().round(2)